# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [42]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [43]:
# connection string
host = 'localhost'
dbname = 'sparkifydb'
user = 'postgres'
password = 'mysecretpassword'

# connect to default database
conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)
cur = conn.cursor()

In [44]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [44]:
song_files = get_files('data/song_data/')
print(len(song_files))

71


In [45]:
filepath = song_files[0]
print(filepath)

/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json


In [46]:
df = pd.read_json(path_or_buf=filepath, typ='series')
df.head(20)

num_songs                            1
artist_id           AR7G5I41187FB4CE6C
artist_latitude                   None
artist_longitude                  None
artist_location        London, England
artist_name                   Adam Ant
song_id             SONHOTT12A8C13493C
title                  Something Girls
duration                       233.404
year                              1982
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [47]:
song_id = df.values[6]
title = df.values[7]
artist_id = df.values[1]
year = df.values[9]
duration = df.values[8]

song_data = (song_id, title, artist_id, year, duration)
song_data
print(song_data)

('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363)


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [48]:
song_table_insert = ("""INSERT INTO songs (song_id, title, artist_id, year, duration)
    VALUES (%s, %s, %s, %s, %s)
""")
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [49]:
artist_id = df.values[1]
name = df.values[5]
location = df.values[4]
latitude = df.values[3]
longitude = df.values[2]

artist_data = (artist_id, name, location, latitude, longitude)
artist_data

('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [50]:
artist_table_insert = ("""INSERT INTO artists (artist_id, name, location, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s)
""")

cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [45]:
log_files = get_files('data/log_data/')
print(log_files)

['/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-11-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-23-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-18-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-04-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-01-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-14-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-08-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-26-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-25-events.json', '/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/l

In [46]:
filepath = log_files[0]
print(filepath)

/Users/deathstar/PycharmProjects/Data-Modeling-Postgres/data/log_data/2018/11/2018-11-11-events.json


In [47]:
df = pd.read_json(path_or_buf=filepath, typ='frame', lines=True)
df.head(20)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
6,Gary Allan,Logged In,Ryann,F,0,Smith,259.83955,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,The One,200,1541930188796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
7,Miracle Fortress,Logged In,Ryann,F,1,Smith,200.90730,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Five Roses,200,1541930447796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
8,Don Omar,Logged In,Ryann,F,2,Smith,261.35465,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Cuentale,200,1541930647796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92
9,Jay-Z,Logged In,Ryann,F,3,Smith,212.27057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,D'Evils,200,1541930908796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [48]:
# Filter for "page":"NextSong"
df.loc[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


In [49]:
# Clean up timestamp column
t = df.loc[df['page']=='NextSong']
t['ts'] = pd.to_datetime(df['ts'], unit='ms')
t.head()

<ipython-input-49-3e905f6f8c7d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['ts'] = pd.to_datetime(df['ts'], unit='ms')


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [106]:
t['t_stamp'] = t['ts'].dt.date
t['t_hour'] = t['ts'].dt.hour
t['t_day'] = t['ts'].dt.day
t['t_week_of_year'] = t['ts'].dt.weekofyear
t['t_month'] = t['ts'].dt.month
t['t_year'] = t['ts'].dt.year
t['t_weekday'] = t['ts'].dt.dayofweek

t.head()

#column_labels = ()

<ipython-input-106-ba5c782f0655>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['t_stamp'] = t['ts'].dt.date
<ipython-input-106-ba5c782f0655>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['t_hour'] = t['ts'].dt.hour
<ipython-input-106-ba5c782f0655>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,t_stamp,t_hour,t_day,t_week_of_year,t_month,t_year,t_weekday
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11,2,11,45,11,2018,6
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11,2,11,45,11,2018,6
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11,2,11,45,11,2018,6
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,...,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11,4,11,45,11,2018,6
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,...,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75,2018-11-11,4,11,45,11,2018,6


In [107]:
time_df = t.filter(items=['t_stamp', 't_hour', 't_day', 't_week_of_year', 't_month', 't_year', 't_weekday'])
time_df.head()

,t_stamp,t_hour,t_day,t_week_of_year,t_month,t_year,t_weekday
0,2018-11-11,2,11,45,11,2018,6
1,2018-11-11,2,11,45,11,2018,6
2,2018-11-11,2,11,45,11,2018,6
4,2018-11-11,4,11,45,11,2018,6
5,2018-11-11,4,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [126]:
time_table_insert = ("""INSERT INTO time (start_time, hour, day, week, month, year, weekday)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
""")

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [117]:
users_df = t.filter(items=['userId', 'firstName', 'lastName', 'gender', 'level'])
users_df

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
...,...,...,...,...,...
90,35,Molly,Taylor,F,free
91,35,Molly,Taylor,F,free
92,35,Molly,Taylor,F,free
93,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [124]:
user_table_insert = ("""INSERT INTO users (user_id, first_name, last_name, gender, level)
    VALUES (%s, %s, %s, %s, %s)
""")

for i, row in users_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.


## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [39]:
df['userAgent'] = df['userAgent'].str.strip(to_strip='"')
df['userAgent'] = df['userAgent'].str.slice(0, 7)
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df['location'] = df['location'].str.split(pat=',', )[0][0]

'Philadelphia-Camden-Wilmington'

In [40]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,Philadelphia-Camden-Wilmington,PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,Mozilla,69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,Philadelphia-Camden-Wilmington,PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,Mozilla,69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,Philadelphia-Camden-Wilmington,PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,Mozilla,69
3,None,Logged In,Lily,F,0,Burns,NaN,free,Philadelphia-Camden-Wilmington,GET,Home,1.540621e+12,456,None,200,2018-11-11 04:34:01.796,Mozilla,32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,Philadelphia-Camden-Wilmington,PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,Mozilla,32


In [41]:
for index, row in df.iterrows():
    artist = row[0]
    song = row[13]
    level = row[7]
    session_id = row[12]
    start_time = row[15]
    user_agent = row[16]
    user_id = row[17]
    location = row[8]

    songplay_table_insert = ("""INSERT INTO songplays (start_time, level, session_id, user_agent, location,
    artist_name, song_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """)

    songplay_data = (start_time, level, session_id, user_agent, location, artist, song)
    print(songplay_data)

    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Frumpies
Fuck Kitty
free
455
2018-11-11 02:33:56.796000
Mozilla
69

(Timestamp('2018-11-11 02:33:56.796000'), 'free', 455, 'Mozilla', 'Philadelphia-Camden-Wilmington', 'Frumpies', 'Fuck Kitty')
Kenny G with Peabo Bryson
By The Time This Night Is Over
free
455
2018-11-11 02:36:10.796000
Mozilla
69

(Timestamp('2018-11-11 02:36:10.796000'), 'free', 455, 'Mozilla', 'Philadelphia-Camden-Wilmington', 'Kenny G with Peabo Bryson', 'By The Time This Night Is Over')
Biffy Clyro
God & Satan
free
455
2018-11-11 02:40:34.796000
Mozilla
69

(Timestamp('2018-11-11 02:40:34.796000'), 'free', 455, 'Mozilla', 'Philadelphia-Camden-Wilmington', 'Biffy Clyro', 'God & Satan')
None
None
free
456
2018-11-11 04:34:01.796000
Mozilla
32

(Timestamp('2018-11-11 04:34:01.796000'), 'free', 456, 'Mozilla', 'Philadelphia-Camden-Wilmington', None, None)
HIM
Beautiful
free
456
2018-11-11 04:36:13.796000
Mozilla
32

(Timestamp('2018-11-11 04:36:13.796000'), 'free', 456, 'Mozilla', 'Philadelphia-Camden-Wilmington', 'HIM

In [ ]:
for index, row in df.iterrows():

#     # get songid and artistid from song and artist tables
#     song_select = ("""INSERT INTO songplays (song_id, artist_id)
#     SELECT songs.song_id, artists.artist_id FROM songs INNER JOIN artists ON songs.artist_id=artists.artist_id;
# """)
#
#     cur.execute(song_select, (row.song, row.artist, row.length))
#     results = cur.fetchone()
#
#     if results:
#         songid, artistid = results
#     else:
#         songid, artistid = None, None

    # insert songplay record
    level = df['level']
    session_id = df['sessionId']
    user_agent = df['userAgent']
    time_stamp = df['ts']

    songplay_table_insert = ("""INSERT INTO songplays (level, session_id, user_agent, timestamp)
        VALUES (%s, %s, %s, %s,)
    """)

    songplay_data = (level, session_id, user_agent, time_stamp)

    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.